In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip -O ml-25m.zip

!unzip ml-25m.zip -d ml-25m

!ls -lh ml-25m

!echo "209172,4993,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,5952,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,7153,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,1035,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,364,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,6539,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,45722,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,3578,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,7458,5" >> ml-25m/ml-25m/ratings.csv
!echo "209172,122896,5" >> ml-25m/ml-25m/ratings.csv


--2025-03-10 01:03:02--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  33.6MB/s    in 8.3s    

2025-03-10 01:03:11 (30.3 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/ml-25m/
  inflating: ml-25m/ml-25m/tags.csv  
  inflating: ml-25m/ml-25m/links.csv  
  inflating: ml-25m/ml-25m/README.txt  
  inflating: ml-25m/ml-25m/ratings.csv  
  inflating: ml-25m/ml-25m/genome-tags.csv  
  inflating: ml-25m/ml-25m/genome-scores.csv  
  inflating: ml-25m/ml-25m/movies.csv  
total 4.0K
drwxrwxr-x 2 root root 4.0K Nov 21  2019 ml-25m


In [ ]:
!rm -rf movie-kompis
!git clone https://github.com/Rajindra/movie-kompis.git


Cloning into 'movie-kompis'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 9 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 5.32 KiB | 5.32 MiB/s, done.
Resolving deltas: 100% (1/1), done.


# New section

In [ ]:
!pip install torch==2.5.1 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pwd
!ls *

/content
ml-25m.zip

ml-25m:
ml-25m

movie-kompis:
Kompis2.py  KompisLarge.py  Kompis.py  README.md  small-data-set

sample_data:
anscombe.json		     california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv		   README.md


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.amp import autocast, GradScaler
import os

# ---- Load Data ----
movies = pd.read_csv('/content/ml-25m/ml-25m/movies.csv')
ratings = pd.read_csv('/content/ml-25m/ml-25m/ratings.csv')
genome_scores = pd.read_csv('/content/ml-25m/ml-25m/genome-scores.csv')

# ---- Map IDs ----
n_users = ratings['userId'].nunique()
n_movies = ratings['movieId'].nunique()

user_map = {id: idx for idx, id in enumerate(ratings['userId'].unique())}
movie_map = {id: idx for idx, id in enumerate(ratings['movieId'].unique())}
reverse_movie_map = {v: k for k, v in movie_map.items()}  # Reverse map for lookup

ratings['userId'] = ratings['userId'].map(user_map)
ratings['movieId'] = ratings['movieId'].map(movie_map)

# ---- Split Data ----
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_users = torch.tensor(train_data['userId'].values, dtype=torch.long)
train_movies = torch.tensor(train_data['movieId'].values, dtype=torch.long)
train_ratings = torch.tensor(train_data['rating'].values, dtype=torch.float)

test_users = torch.tensor(test_data['userId'].values, dtype=torch.long)
test_movies = torch.tensor(test_data['movieId'].values, dtype=torch.long)
test_ratings = torch.tensor(test_data['rating'].values, dtype=torch.float)

# ---- GPU Support ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- DataLoader ----
batch_size = 1024
train_dataset = TensorDataset(train_users, train_movies, train_ratings)
test_dataset = TensorDataset(test_users, test_movies, test_ratings)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ---- Model ----
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=64):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(n_users + 1, n_factors)
        self.movie_embedding = nn.Embedding(n_movies, n_factors)
        self.bias_user = nn.Embedding(n_users + 1, 1)
        self.bias_movie = nn.Embedding(n_movies, 1)
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, user, movie):
        user_embedding = self.user_embedding(user)
        movie_embedding = self.movie_embedding(movie)
        pred = (user_embedding * movie_embedding).sum(1)
        pred += self.bias_user(user).squeeze() + self.bias_movie(movie).squeeze() + self.bias
        return pred

# ---- Check if the model file exists ----
model_filename = 'movie_recommender_model.pth'
model = MatrixFactorization(n_users, n_movies).to(device)

# If the model already exists, load it
if os.path.exists(model_filename):
    print("Loading saved model...")
    model.load_state_dict(torch.load(model_filename))
    model.eval()  # Set the model to evaluation mode
else:
    print("Training a new model...")

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    scaler = GradScaler()

    # ---- Train Model ----
    n_epochs = 20
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0

        for users, movies, ratings in train_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)

            optimizer.zero_grad()
            with autocast('cuda'):
                predictions = model(users, movies)
                loss = loss_fn(predictions, ratings)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch}: Loss = {avg_loss:.4f}')

    # ---- Save Model after training ----
    torch.save(model.state_dict(), model_filename)
    print(f"Model saved to {model_filename}")

# ---- Evaluate Model ----
model.eval()
with torch.no_grad():
    total_loss = 0
    for users, movies, ratings in test_loader:
        users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
        predictions = model(users, movies)
        loss = loss_fn(predictions, ratings)
        total_loss += loss.item()

    test_loss = total_loss / len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')

# ---- Precompute Genre Weights (filtering out tags with low relevance) ----
# Group by movieId and only include tags with relevance greater than 0.8
movie_genre_weights = genome_scores.groupby('movieId', group_keys=False).apply(
    lambda x: {
        row['tagId']: row['relevance'] for _, row in x.iterrows() if row['relevance'] > 0.8
    }
).to_dict()

def get_favorite_genre_weights(favorite_movies):
    genre_weights = {}
    for movie in favorite_movies:
        if movie in movie_map:
            genres = movie_genre_weights.get(movie_map[movie], {})
            for tag, relevance in genres.items():
                genre_weights[tag] = genre_weights.get(tag, 0) + relevance

    total_weight = sum(genre_weights.values())
    if total_weight > 0:
        for tag in genre_weights:
            genre_weights[tag] /= total_weight

    return genre_weights

# ---- Recommend Movies ----
def recommend_movies(user_favorites, n=5):
    favorite_movieIds = [movie_map[movie] for movie in user_favorites if movie in movie_map]
    genre_weights = get_favorite_genre_weights(user_favorites)

    scores = []
    with torch.no_grad():
        for movieId in range(n_movies):
            if movieId not in favorite_movieIds:
                # Convert Tensor to integer using .item()
                pred = model(
                    torch.tensor([n_users], device=device),
                    torch.tensor([movieId], device=device)
                ).item()

                # Fix indexing issue with genre weights
                movie_genre = movie_genre_weights.get(reverse_movie_map.get(movieId), {})
                genre_score = sum(
                    genre_weights.get(tag, 0) * relevance for tag, relevance in movie_genre.items()
                )

                # ✅ Convert movieId to int before adding to scores
                scores.append((int(movieId), pred + genre_score))

    # Sort by predicted score + genre adjustment
    scores.sort(key=lambda x: x[1], reverse=True)

    # ✅ Convert Tensor to integer before lookup
    # Rename the movies DataFrame to df_movies to avoid conflicts
    recommendations = [
        df_movies.loc[df_movies['movieId'] == reverse_movie_map[x[0]], 'title'].values[0]
        for x in scores[:n]
    ]
    return recommendations

# ---- Example Usage ----
# Rename the movies DataFrame to df_movies to avoid conflicts
df_movies = pd.read_csv('/content/ml-25m/ml-25m/movies.csv')
user_favorites = [4993, 5952, 7153, 1035, 364, 6539, 45722, 3578, 7458, 122896]
suggestions = recommend_movies(user_favorites)
print("Recommended Movies:", suggestions)


Training a new model...
Epoch 0: Loss = 21.7715
Epoch 1: Loss = 2.5469
Epoch 2: Loss = 1.1645
Epoch 3: Loss = 0.8729
Epoch 4: Loss = 0.7565
Epoch 5: Loss = 0.6917
Epoch 6: Loss = 0.6476
Epoch 7: Loss = 0.6127
Epoch 8: Loss = 0.5837
Epoch 9: Loss = 0.5584
Epoch 10: Loss = 0.5360
Epoch 11: Loss = 0.5157
Epoch 12: Loss = 0.4974
Epoch 13: Loss = 0.4806
Epoch 14: Loss = 0.4654
Epoch 15: Loss = 0.4518
Epoch 16: Loss = 0.4394
Epoch 17: Loss = 0.4283
Epoch 18: Loss = 0.4182
Epoch 19: Loss = 0.4092
Model saved to movie_recommender_model.pth
Test Loss: 0.9420


<ipython-input-5-3aa1f6dd6f28>:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  movie_genre_weights = genome_scores.groupby('movieId', group_keys=False).apply(


Recommended Movies: ['Dead Lucky (2017)', 'Every Stewardess Goes to Heaven (Todas las azafatas van al cielo) (2002)', 'Make-Out with Violence (2008)', 'Dark Future (1994)', 'Shatter (1974)']
